In [7]:
import boto3
import pandas as pd

In [8]:
dynamo_client = boto3.resource('dynamodb',
                    region_name='us-west-2')
    
table = dynamo_client.Table('raw_weather_data')

In [9]:
response = table.scan()
weather_conditions = response['Items']
weather_conditions


[{'feels_like': '9.24',
  'max_temp': '11.33',
  'name': 'Liège',
  'temperature': '10.73',
  'dt': Decimal('1741337051'),
  'longitude': '5.5675',
  'commune': 'Liège',
  'min_temp': '10.62',
  'humidity': '53',
  'description': 'clear sky',
  'latitude': '50.6337',
  'cloud_pct': '0',
  'weather': 'Clear',
  'wind_speed': Decimal('18.5')},
 {'feels_like': '10.87',
  'max_temp': '11.9',
  'name': 'Liège',
  'temperature': '11.9',
  'dt': Decimal('1741337232'),
  'longitude': '5.5675',
  'commune': 'Liège',
  'min_temp': '11.9',
  'humidity': '66',
  'description': 'broken clouds',
  'latitude': '50.6337',
  'cloud_pct': '59',
  'weather': 'Clouds',
  'wind_speed': Decimal('11.74')},
 {'feels_like': '9.42',
  'max_temp': '12.88',
  'name': 'Liège',
  'temperature': '10.89',
  'dt': Decimal('1741337593'),
  'longitude': '5.5675',
  'commune': 'Liège',
  'min_temp': '10.72',
  'humidity': '53',
  'description': 'clear sky',
  'latitude': '50.6337',
  'cloud_pct': '0',
  'weather': 'Clear

In [10]:
df = pd.DataFrame(weather_conditions)
df.head()

,feels_like,max_temp,name,temperature,dt,longitude,commune,min_temp,humidity,description,latitude,cloud_pct,weather,wind_speed
0,9.24,11.33,Liège,10.73,1741337051,5.5675,Liège,10.62,53,clear sky,50.6337,0,Clear,18.5
1,10.87,11.9,Liège,11.9,1741337232,5.5675,Liège,11.9,66,broken clouds,50.6337,59,Clouds,11.74
2,9.42,12.88,Liège,10.89,1741337593,5.5675,Liège,10.72,53,clear sky,50.6337,0,Clear,18.5
3,10.32,12.88,Liège,11.78,1741338315,5.5675,Liège,11.72,50,clear sky,50.6337,0,Clear,22.21
4,10.25,12.28,Liège,11.72,1741338856,5.5675,Liège,11.72,50,clear sky,50.6337,0,Clear,22.21


In [11]:
# Transform the data stores as string to numerical

df['temperature'] = df['temperature'].astype(float)
df['min_temp'] = df['min_temp'].astype(float)
df['max_temp'] = df['max_temp'].astype(float)
df['feels_like'] = df['feels_like'].astype(float)
df['longitude'] = df['longitude'].astype(float)
df['latitude'] = df['latitude'].astype(float)
df['humidity'] = df['humidity'].astype(int)
df['cloud_pct'] = df['cloud_pct'].astype(int)
df['wind_speed'] = df['wind_speed'].astype(float)

In [12]:
# Let's do some Exploratory Data Analysis

grouped_df = df.groupby('name')
new_df = grouped_df['temperature'].mean().round(1)
print('Average temperatures:')
new_df

Average temperatures:


name
Antwerp      12.3
Bruges       11.8
Brussels     12.0
Charleroi    11.1
Ghent        11.9
Leuven       12.3
Liège        12.2
Mons         11.6
Namur        11.8
Ostend       10.8
Name: temperature, dtype: float64

In [13]:
warmest_city = grouped_df['temperature'].mean().round(2).sort_values(ascending=False).head(1)
print(f'The warmest city at the moment is: {warmest_city.index[0]} with {warmest_city.values[0]} degrees celcius.')



The warmest city at the moment is: Leuven with 12.31 degrees celcius.


In [14]:
chilliest_city = grouped_df['temperature'].mean().round(2).sort_values().head(1)
print(f'The chilliest city at the moment is: {chilliest_city.index[0]} with {chilliest_city.values[0]} degrees celcius.')


The chilliest city at the moment is: Ostend with 10.83 degrees celcius.


In [15]:
# Check how many entries there are per city:

scores = dict()

for item in weather_conditions:
    for key, val in item.items():
        if key == 'name':
            city = val
            if city not in scores.keys():
                scores.update({city: 1})
            else:
                number = scores.get(city)
                number += 1
                scores.update({f'{city}' : number})

for city in scores.items():
    print(city)
print(f'\n Total records: {len(weather_conditions)}')


('Liège', 9)
('Antwerp', 9)
('Ghent', 7)
('Namur', 8)
('Brussels', 9)
('Ostend', 6)
('Bruges', 9)
('Charleroi', 7)
('Mons', 8)
('Leuven', 7)

 Total records: 79
